In [5]:
import pandas as pd

#import numpy as np
#from numpy import genfromtxt, savetxt

#import random
#from random import sample

#import matplotlib.pyplot as plt

#from sklearn.decomposition import PCA

import xgboost as xgb

#from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, cross_val_score

#from sklearn.grid_search import GridSearchCV

#from sklearn.metrics import accuracy_score

import csv
import numpy as np
from numpy import arange

In [2]:
date_lst= ['Id','L3_S30_D3496', 'L3_S30_D3506',
             'L3_S30_D3501', 'L3_S30_D3516',
             'L3_S30_D3511']

numorg=['L1_S24_F1604',
                      'L1_S24_F1695', 
                      'L1_S24_F1846', 
                      'L3_S29_F3407', 
                      'L3_S32_F3850',
                      'L3_S33_F3855', 
                      'L3_S33_F3865']

In [3]:
path_d_tr='Y://Data//Kaggle//bosch//train_date//train_date.csv'
path_d_te='Y://Data//Kaggle//bosch//test_date//test_date.csv'

In [8]:
numcol=['Id', 'L3_S30_F3754', 'L3_S36_F3920', 'L3_S33_F3857', 'L3_S33_F3859', 'L3_S30_F3744', 'L0_S0_F20', 'L3_S33_F3865',
        'L3_S30_F3759', 'L3_S30_F3749', 'L3_S30_F3809', 'L0_S1_F28', 'L3_S29_F3479', 'L3_S30_F3774', 'L3_S30_F3554', 
        'L3_S30_F3804', 'L3_S30_F3769', 'L3_S29_F3339', 'L3_S30_F3829', 'L3_S30_F3574', 'L3_S32_F3850', 'L1_S24_F1723', 
        'L3_S29_F3407', 'L3_S29_F3345', 'L3_S29_F3348', 'L3_S29_F3351', 'L3_S38_F3952', 'L3_S35_F3889', 'L3_S29_F3327',
        'L3_S30_F3604', 'L3_S38_F3960', 'L3_S29_F3333', 'L3_S30_F3704', 'L3_S30_F3494', 'L3_S35_F3896', 'L3_S29_F3324',
        'L3_S29_F3373', 'L3_S30_F3609', 'L3_S29_F3379', 'L3_S30_F3639', 'L0_S1_F24', 'L2_S26_F3106', 'L3_S29_F3321', 
        'L3_S38_F3956', 'L0_S0_F16', 'L3_S29_F3336', 'L3_S30_F3534', 'L1_S24_F1846', 'L3_S29_F3427','L1_S24_F1604',
        'L1_S24_F1695', 'L3_S33_F3855' ]

In [3]:
len(numcol)

52

In [3]:
def read_numtrain(path_num, path_d_tr, cols):
    X_all_train=pd.read_csv(path_d_tr)
    X_train_ctg=X_all_train.dropna(axis=1, how='all')
    response=pd.read_csv(path_num, usecols=['Response'])
    X_num = pd.read_csv(path_num, usecols=cols)
    X_train=pd.merge(X_num, X_train_ctg,on='Id')
    
    return X_train, response

In [13]:
def num_feature(X_train,response,numcol):
    column_name=X_train.columns
    
    grid = {'max_depth':10,
        #'booster': 'gbtree',
        'objective': 'binary:logistic',
        'subsample': 0.8,
        'colsample_bytree':0.8,
        'learning_rate': 0.1,
        'seed': 2016,
        'silent': 0,
        #'gamma':0.05
           }

    clf1 = xgb.XGBClassifier()
    clf1.set_params(**grid)

    clf1.fit(X_train,response['Response'])
    
    importance=clf1.feature_importances_
    
    importance_s=sorted([(index,value) for index, value in enumerate(importance.tolist())],key=lambda x: x[1],reverse = True)
    
    #important_feature_index=[index for (index,val) in importance_s if val>0.0035]
    
    col_name = [(column_name[index],val) for (index,val) in importance_s if column_name[index] not in numcol]
    #filter the most important featrues for date data(30,25,20)
    print('30',[column_name[index] for (index,val) in importance_s if column_name[index] not in numcol][:30])
    
    print('25',[column_name[index] for (index,val) in importance_s if column_name[index] not in numcol][:25])
    
    print('20',[column_name[index] for (index,val) in importance_s if column_name[index] not in numcol][:20])
    
    return col_name

In [10]:
def readbychunk(path_d, path_num,cols):
    size=600000
    for tr_d, tr_n, res in zip(pd.read_csv(path_d, chunksize=size), pd.read_csv(path_num, usecols=cols,chunksize=size),
                       pd.read_csv(path_num, usecols=['Response'], chunksize=size)):
        
        #new_X_train_ctg=tr.dropna(axis=1,how='all')
        #pd_ctg=pd.get_dummies(new_X_train_ctg)
        
        train=pd.merge(tr_d, tr_n,on='Id')
        print('############Chunk train ready#############')

        cols=num_feature(train, res,cols)
        print('All',cols)

In [14]:
path_d='Y://Data//Kaggle//bosch//train_date//train_date.csv'
path_num='Y://Data//Kaggle//bosch//train_numeric//train_numeric.csv'

readbychunk(path_d, path_num,numcol)

############Chunk train ready#############
30 ['L3_S36_D3919', 'L3_S38_D3953', 'L3_S33_D3856', 'L3_S35_D3886', 'L3_S29_D3474', 'L2_S26_D3037', 'L0_S3_D70', 'L3_S34_D3875', 'L0_S7_D137', 'L0_S10_D216', 'L0_S4_D106', 'L0_S9_D152', 'L2_S27_D3130', 'L0_S5_D115', 'L0_S2_D34', 'L0_S11_D280', 'L0_S17_D432', 'L1_S24_D1765', 'L3_S32_D3852', 'L0_S6_D120', 'L0_S15_D395', 'L0_S0_D1', 'L0_S12_D331', 'L3_S29_D3316', 'L0_S16_D423', 'L1_S24_D1583', 'L0_S22_D543', 'L0_S4_D111', 'L1_S24_D1809', 'L0_S18_D437']
25 ['L3_S36_D3919', 'L3_S38_D3953', 'L3_S33_D3856', 'L3_S35_D3886', 'L3_S29_D3474', 'L2_S26_D3037', 'L0_S3_D70', 'L3_S34_D3875', 'L0_S7_D137', 'L0_S10_D216', 'L0_S4_D106', 'L0_S9_D152', 'L2_S27_D3130', 'L0_S5_D115', 'L0_S2_D34', 'L0_S11_D280', 'L0_S17_D432', 'L1_S24_D1765', 'L3_S32_D3852', 'L0_S6_D120', 'L0_S15_D395', 'L0_S0_D1', 'L0_S12_D331', 'L3_S29_D3316', 'L0_S16_D423']
20 ['L3_S36_D3919', 'L3_S38_D3953', 'L3_S33_D3856', 'L3_S35_D3886', 'L3_S29_D3474', 'L2_S26_D3037', 'L0_S3_D70', 'L3_S34_D387

In [21]:
d2=['L3_S36_D3919',
 'L3_S35_D3886',
 'L3_S29_D3474',
 'L3_S38_D3953',
 'L2_S27_D3130',
 'L3_S33_D3856',
 'L2_S26_D3037',
 'L3_S29_D3316',
 'L3_S34_D3875',
 'L0_S4_D106',
 'L0_S6_D120',
 'L0_S7_D137',
 'L0_S2_D34',
 'L0_S11_D280',
 'L0_S9_D152',
 'L0_S3_D70',
 'L0_S5_D115',
 'L0_S10_D216',
 'L3_S32_D3852',
 'L0_S0_D1']

d1=['L3_S36_D3919', 'L3_S38_D3953', 'L3_S33_D3856', 'L3_S35_D3886', 'L3_S29_D3474', 'L2_S26_D3037', 'L0_S3_D70', 
    'L3_S34_D3875', 'L0_S7_D137', 'L0_S10_D216', 'L0_S4_D106', 'L0_S9_D152', 'L2_S27_D3130', 'L0_S5_D115', 'L0_S2_D34', 
    'L0_S11_D280', 'L0_S17_D432', 'L1_S24_D1765', 'L3_S32_D3852', 'L0_S6_D120','L3_S29_D3316','L0_S0_D1']

print(len(set(d1) | set(d2) ))
print(len(d1))

22
22


In [16]:
tup= [('Id', 0.028037382289767265), ('L3_S33_F3857', 0.02157779037952423), ('L0_S0_F20', 0.02109675668179989), 
      ('L3_S30_F3809', 0.02102803811430931), ('L3_S29_F3327', 0.02020340785384178), ('L3_S30_F3774', 0.01999725215137005), 
      ('L3_S33_F3859', 0.01965365558862686), ('L3_S30_F3754', 0.0193787794560194), ('L3_S36_F3920', 0.018966464325785637),
      ('L3_S30_F3759', 0.018760308623313904), ('L3_S30_F3744', 0.0180043987929821), ('L3_S33_F3865', 0.017179768532514572), 
      ('L3_S30_F3749', 0.016904892399907112), ('L3_S30_F3554', 0.01614898256957531), ('L3_S30_F3494', 0.01587410643696785),
      ('L3_S29_F3333', 0.015667948871850967), ('L3_S35_F3889', 0.015667948871850967), ('L3_S30_F3804', 0.014637162908911705), 
      ('L3_S29_F3479', 0.014224848710000515), ('L3_S29_F3336', 0.014156130142509937), ('L3_S30_F3609', 0.014018691144883633),
      ('L3_S30_F3769', 0.013881253078579903), ('L3_S29_F3348', 0.013606376945972443), ('L3_S29_F3345', 0.013331500813364983),
      ('L3_S30_F3604', 0.0131253432482481), ('L3_S30_F3574', 0.012781748548150063), ('L2_S26_F3106', 0.01271302904933691),
      ('L0_S0_F16', 0.01257559098303318), ('L3_S36_D3919', 0.012506871484220028), ('L3_S29_F3373', 0.01243815291672945),
      ('L3_S30_F3639', 0.012369433417916298), ('L3_S29_F3339', 0.011957119219005108), ('L3_S29_F3379', 0.011957119219005108), 
      ('L3_S30_F3829', 0.01188840065151453), ('L3_S30_F3534', 0.011819681152701378), ('L0_S1_F28', 0.011613523587584496), 
      ('L3_S29_F3351', 0.011407366953790188), ('L3_S35_F3896', 0.011407366953790188), ('L0_S1_F24', 0.011269928887486458), 
      ('L3_S35_D3886', 0.011132490821182728), ('L3_S29_D3474', 0.010032985359430313), ('L3_S38_D3953', 0.009758109226822853),
      ('L3_S29_F3321', 0.0096893897280097), ('L3_S29_F3427', 0.009620671160519123), ('L3_S30_F3704', 0.00955195166170597), 
      ('L1_S24_F1723', 0.00941451359540224), ('L3_S29_F3324', 0.00927707552909851), ('L2_S27_D3130', 0.00913963746279478), 
      ('L3_S33_D3856', 0.00900219939649105), ('L2_S26_D3037', 0.008589884266257286), ('L3_S29_D3316', 0.008040132001042366), 
      ('L1_S24_F1846', 0.007971412502229214), ('L3_S34_D3875', 0.007902693934738636), ('L0_S4_D106', 0.007765255402773619), 
      ('L0_S6_D120', 0.007559098303318024), ('L3_S32_F3850', 0.007284222170710564), ('L0_S7_D137', 0.007146784104406834),
      ('L0_S2_D34', 0.007009345572441816), ('L0_S11_D280', 0.006803188472986221), ('L0_S9_D152', 0.006459593307226896),
      ('L0_S3_D70', 0.006184716708958149), ('L0_S5_D115', 0.006184716708958149), ('L0_S10_D216', 0.005978559609502554), 
      ('L3_S32_D3852', 0.005566245410591364), ('L3_S38_F3960', 0.005360087845474482), ('L0_S0_D1', 0.004741616081446409),
      ('L0_S14_D360', 0.004604178015142679), ('L0_S12_D331', 0.004466739948838949), ('L0_S19_D454', 0.004398020915687084), 
      ('L0_S23_D617', 0.004398020915687084), ('L3_S30_D3496', 0.004329301882535219), ('L0_S17_D432', 0.004191863816231489), 
      ('L1_S24_D1765', 0.004191863816231489), ('L3_S38_F3956', 0.003779549151659012), ('L0_S18_D437', 0.003710830118507147), 
      ('L0_S16_D423', 0.003435953753069043), ('L3_S30_D3566', 0.003367234719917178), ('L0_S15_D395', 0.003298515686765313), 
      ('L3_S33_F3855', 0.003161077620461583), ('L1_S24_D1583', 0.0029549202881753445), ('L1_S24_F1604', 0.0028862012550234795),
      ('L0_S21_D469', 0.002680043922737241), ('L2_S26_D3041', 0.002680043922737241), ('L3_S35_D3891', 0.002611324889585376), 
      ('L1_S24_D818', 0.0024051677901297808), ('L0_S22_D543', 0.002199010457843542), ('L1_S24_D1809', 0.002199010457843542), 
      ('L3_S33_D3858', 0.002199010457843542), ('L0_S5_D117', 0.002130291424691677), ('L3_S36_D3921', 0.002130291424691677), 
      ('L0_S3_D74', 0.002061572391539812), ('L0_S9_D157', 0.002061572391539812), ('L3_S37_D3942', 0.002061572391539812), 
      ('L1_S25_D2098', 0.0019928531255573034), ('L3_S29_D3477', 0.0019928531255573034), ('L0_S4_D111', 0.0019241340924054384),
      ('L1_S24_F1695', 0.0019241340924054384), ('L0_S8_D145', 0.0018554150592535734), ('L1_S24_D1558', 0.0018554150592535734),
      ('L1_S24_D1674', 0.0017866960261017084), ('L1_S25_D1980', 0.0017866960261017084), ('L2_S28_D3223', 0.0017866960261017084),
      ('L0_S21_D474', 0.0017179768765345216), ('L3_S31_D3836', 0.0016492578433826566), ('L1_S24_D999', 0.0015805388102307916), 
      ('L1_S24_D1511', 0.0015805388102307916), ('L1_S25_D1902', 0.0015805388102307916), ('L2_S27_D3134', 0.0015805388102307916),
      ('L3_S34_D3877', 0.0015118196606636047), ('L0_S1_D26', 0.0014431006275117397), ('L1_S24_D1277', 0.0014431006275117397), 
      ('L0_S2_D38', 0.001374381477944553), ('L0_S14_D364', 0.001374381477944553), ('L0_S23_D621', 0.001374381477944553), 
      ('L3_S29_D3319', 0.001374381477944553), ('L1_S24_D677', 0.001305662444792688), ('L1_S24_D1770', 0.001305662444792688),
      ('L3_S29_F3407', 0.001305662444792688), ('L0_S11_D284', 0.0012369434116408229), ('L3_S38_D3957', 0.0012369434116408229),
      ('L0_S16_D428', 0.001168224262073636), ('L1_S24_D1155', 0.001168224262073636), ('L0_S10_D221', 0.001099505228921771), 
      ('L1_S24_D1368', 0.001099505228921771), ('L3_S49_D4208', 0.001099505228921771), ('L3_S38_F3952', 0.001099505228921771), 
      ('L0_S7_D139', 0.001030786195769906), ('L1_S25_D2329', 0.0009620670462027192), ('L3_S30_D3571', 0.0009620670462027192)]

In [18]:
[name for (name,val) in tup if name not in numcol][:20]

['L3_S36_D3919',
 'L3_S35_D3886',
 'L3_S29_D3474',
 'L3_S38_D3953',
 'L2_S27_D3130',
 'L3_S33_D3856',
 'L2_S26_D3037',
 'L3_S29_D3316',
 'L3_S34_D3875',
 'L0_S4_D106',
 'L0_S6_D120',
 'L0_S7_D137',
 'L0_S2_D34',
 'L0_S11_D280',
 'L0_S9_D152',
 'L0_S3_D70',
 'L0_S5_D115',
 'L0_S10_D216',
 'L3_S32_D3852',
 'L0_S0_D1']

In [5]:
def main():
    path_d_tr='Y://Data//Kaggle//bosch//train_date//train_date.csv'
    path_num='Y://Data//Kaggle//bosch//train_numeric//train_numeric.csv'
    X_train,response=read_numtrain(path_num,path_d_tr,numcol)
    print("######################## data ready ##############################")
    cols=num_feature(X_train,response,numcol)
    print(cols)

In [ ]:
main()